In [4]:
# API Dokumentationen:
# https://bmrs.elexon.co.uk/api-documentation/endpoint/forecast/
# https://api.rebase.energy/weather/docs/v2/#general-usage
# https://api.rebase.energy/challenges/redoc#tag/

In [1]:
import comp_utils
rebase_api_client = comp_utils.RebaseAPI(api_key = open("DanMaLeo_key.txt").read())

In [8]:
import requests, json
url = f"https://api.rebase.energy/challenges/data/day_ahead_demand"
resp = requests.get(url)
print(resp)
resp.json()

<Response [401]>


'Bearer token is missing'

In [ ]:
https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/weekly/history?publishTime=2024-01-01&range=52W&format=csv

In [4]:
import requests, json
url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/margin/weekly/history?publishTime=2024-01-01&range=52W&format=csv"
resp = requests.get(url)
print(resp)
resp.json()

<Response [200]>

In [43]:
from datetime import date

all_api_data = list()

for year in range(2020, 2025, 1):
    start_date = date(year, 5, 1) 
    print(str(start_date))

    import csv

    name = "margin"

    url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/{name}/weekly/history?publishTime={start_date}&range=52W&format=csv"

    with requests.Session() as s:
        download = s.get(url)

        decoded_content = download.content.decode('utf-8')

        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        all_api_data.append(my_list)

2020-05-01
2021-05-01
2022-05-01
2023-05-01
2024-05-01


In [41]:
from datetime import date

all_surplus_api_data = list()

for year in range(2020, 2025, 1):
    start_date = date(year, 5, 1) 
    print(str(start_date))

    import csv

    name = "surplus"

    url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/{name}/weekly/history?publishTime={start_date}&range=52W&format=csv"

    with requests.Session() as s:
        download = s.get(url)

        decoded_content = download.content.decode('utf-8')

        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        all_surplus_api_data.append(my_list)

2020-09-01
2021-09-01
2022-09-01
2023-09-01
2024-09-01


In [112]:
import pandas as pd
df = pd.concat([pd.DataFrame(x) for x in all_api_data])
df.columns = df.iloc[0]
df = df[df["Week"] != "Week"]
df["WeekStartDate"] = pd.to_datetime(df["WeekStartDate"])

expanded_df = pd.DataFrame()

for week_start in df['WeekStartDate']:
    # Generate a range of dates for the week (7 days starting from week_start)
    week_dates = pd.date_range(start=week_start, periods=7, freq='D')
    
    # Create a DataFrame for the current week and append to the expanded DataFrame
    expanded_df = pd.concat([expanded_df, pd.DataFrame({'date': week_dates})])

# Reset index for a clean DataFrame
expanded_df.reset_index(drop=True, inplace=True)

df = expanded_df.merge(df, left_on = "date", right_on = "WeekStartDate", how = "left").ffill()
df.drop(["WeekStartDate", "Year", "PublishTime"], axis = 1, inplace = True)
df.head()

,date,Week,Margin
0,2021-05-10,19,11957
1,2021-05-11,19,11957
2,2021-05-12,19,11957
3,2021-05-13,19,11957
4,2021-05-14,19,11957


In [113]:
import pandas as pd
df = pd.concat([pd.DataFrame(x) for x in all_surplus_api_data])
df.columns = df.iloc[0]
df = df[df["Week"] != "Week"]
df["WeekStartDate"] = pd.to_datetime(df["WeekStartDate"])

expanded_df = pd.DataFrame()

for week_start in df['WeekStartDate']:
    # Generate a range of dates for the week (7 days starting from week_start)
    week_dates = pd.date_range(start=week_start, periods=7, freq='D')
    
    # Create a DataFrame for the current week and append to the expanded DataFrame
    expanded_df = pd.concat([expanded_df, pd.DataFrame({'date': week_dates})])

# Reset index for a clean DataFrame
expanded_df.reset_index(drop=True, inplace=True)

df = expanded_df.merge(df, left_on = "date", right_on = "WeekStartDate", how = "left").ffill()
df.drop(["WeekStartDate", "Year", "PublishTime"], axis = 1, inplace = True)
df.head()

,date,Week,Surplus
0,2021-09-13,37,6897
1,2021-09-14,37,6897
2,2021-09-15,37,6897
3,2021-09-16,37,6897
4,2021-09-17,37,6897


In [156]:
from datetime import date
import csv

def preprocess_margin_or_surplus_data(data):
    df = pd.concat([pd.DataFrame(x) for x in data])
    df.columns = df.iloc[0]
    df = df[df["Week"] != "Week"]
    df["WeekStartDate"] = pd.to_datetime(df["WeekStartDate"])

    expanded_df = pd.DataFrame()

    for week_start in df['WeekStartDate']:
        # Generate a range of dates for the week (7 days starting from week_start)
        week_dates = pd.date_range(start=week_start, periods=7, freq='D')
        
        # Create a DataFrame for the current week and append to the expanded DataFrame
        expanded_df = pd.concat([expanded_df, pd.DataFrame({'date': week_dates})])

    # Reset index for a clean DataFrame
    expanded_df.reset_index(drop=True, inplace=True)
    df = expanded_df.merge(df, left_on = "date", right_on = "WeekStartDate", how = "left").ffill()
    df.drop(["WeekStartDate", "Year", "PublishTime"], axis = 1, inplace = True)

    return df

def preprocess_recent_margin_or_surplus_data(df):
    df = pd.DataFrame(df)
    df.columns = df.iloc[0]
    df = df[df["PublishTime"] != "PublishTime"]
    df["PublishTime"] = pd.to_datetime(df["PublishTime"])
    df["ForecastDate"] = pd.to_datetime(df["ForecastDate"])


    expanded_df = pd.DataFrame()

    for week_start in df['ForecastDate']:
        # Generate a range of dates for the week (7 days starting from week_start)
        week_dates = pd.date_range(start=week_start, periods=7, freq='D')
        
        # Create a DataFrame for the current week and append to the expanded DataFrame
        expanded_df = pd.concat([expanded_df, pd.DataFrame({'date': week_dates})])

    # Reset index for a clean DataFrame
    expanded_df.reset_index(drop=True, inplace=True)
    df = expanded_df.merge(df, left_on = "date", right_on = "ForecastDate", how = "left").ffill()
    df.drop(["ForecastDate", "PublishTime"], axis = 1, inplace = True)

    return df


def extract_margin_or_surplus_data(name:str, start_year:int = 2020):

    all_api_data = list()
    name = name.lower()

    for year in range(start_year, 2025, 1):
        start_date = date(year, 5, 1) 

        url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/{name}/weekly/history?publishTime={start_date}&range=52W&format=csv"

        with requests.Session() as s:
            download = s.get(url)

            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            all_api_data.append(my_list)

    df = preprocess_margin_or_surplus_data(all_api_data)
    
    return df


def extract_recent_margin_or_surplus_data(name:str):
    name = name.lower()

    url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/{name}/daily?format=csv"

    with requests.Session() as s:
        download = s.get(url)

        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

    df = preprocess_recent_margin_or_surplus_data(my_list)
    
    return df


def combine_margin_and_surplus_data():
    df_margin = extract_margin_or_surplus_data("margin")
    df_surplus = extract_margin_or_surplus_data("surplus")

    return df_margin.merge(df_surplus, on = ["date", "Week"])

df = combine_margin_and_surplus_data()


def combine_recent_margin_and_surplus_data():
    df_margin = extract_recent_margin_or_surplus_data("margin")
    df_surplus = extract_recent_margin_or_surplus_data("surplus")

    return df_margin.merge(df_surplus, on = ["date", "Week"])

df = combine_margin_and_surplus_data()


In [157]:
def combine_margin_and_surplus_data():
    df_margin = extract_margin_or_surplus_data("margin")
    df_surplus = extract_margin_or_surplus_data("surplus")

    return df_margin.merge(df_surplus, on = ["date", "Week"])

df = combine_margin_and_surplus_data()

In [158]:
def combine_recent_margin_and_surplus_data():
    df_margin = extract_recent_margin_or_surplus_data("margin")
    df_surplus = extract_recent_margin_or_surplus_data("surplus")

    return df_margin.merge(df_surplus, on = ["date", "Week"])

df = combine_margin_and_surplus_data()

In [159]:
df

,date,Week,Margin,Surplus
0,2021-05-10,19,11957,7970
1,2021-05-11,19,11957,7970
2,2021-05-12,19,11957,7970
3,2021-05-13,19,11957,7970
4,2021-05-14,19,11957,7970
...,...,...,...,...
1423,2025-04-30,18,10153,6364
1424,2025-05-01,18,10153,6364
1425,2025-05-02,18,10153,6364
1426,2025-05-03,18,10153,6364


In [2]:
import requests, json
import pandas as pd
url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/daily"
resp = requests.get(url, params = {"forecastDate": "2024-10-28"})
print(resp)
resp.json()

<Response [200]>


{'metadata': {'datasets': ['OCNMFD']},
 'data': [{'forecastDate': '2024-10-30',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 1254},
  {'forecastDate': '2024-10-31',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 7737},
  {'forecastDate': '2024-11-01',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 3631},
  {'forecastDate': '2024-11-02',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 5711},
  {'forecastDate': '2024-11-03',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 4085},
  {'forecastDate': '2024-11-04',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 1094},
  {'forecastDate': '2024-11-05',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 1033},
  {'forecastDate': '2024-11-06',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 542},
  {'forecastDate': '2024-11-07',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 3433},
  {'forecastDate': '2024-11-08',
   'publishTime': '2024-10-28T07:20:00Z',
   'surplus': 6

In [29]:
rebase_api_client.get_margin_forecast()

<Response [200]>


[{'forecast_date': '2024-10-27',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 12347.0},
 {'forecast_date': '2024-10-28',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 5932.0},
 {'forecast_date': '2024-10-29',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 5518.0},
 {'forecast_date': '2024-10-30',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 6825.0},
 {'forecast_date': '2024-10-31',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 9443.0},
 {'forecast_date': '2024-11-01',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 8815.0},
 {'forecast_date': '2024-11-02',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 11668.0},
 {'forecast_date': '2024-11-03',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 10783.0},
 {'forecast_date': '2024-11-04',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 5370.0},
 {'forecast_date': '2024-11-05',
  'publish_time_utc': '2024-10-25T11:00:00Z',
  'margin': 7182.0

In [26]:
import requests, json
import pandas as pd
url = f"https://data.elexon.co.uk/bmrs/api/v1/forecast/surplus/daily/history?forecastDate=2024-08-01"
resp = requests.get(url)
resp.json()

{'statusCode': 404, 'message': 'Resource not found'}

In [16]:
test_date = "2024-10-10"
#rebase_api_client.get_variable(day=test_date,variable="imbalance_price").head()
rebase_api_client.get_variable(day=test_date,variable="wind_total_production").head()
#rebase_api_client.get_variable(day=test_date,variable="solar_total_production").head()

,timestamp_utc,generation_mw,installed_capacity_mwp,capacity_mwp
0,2024-10-09T22:00:00Z,0.0,2956.745251,2779.412861
1,2024-10-09T22:30:00Z,0.0,2956.745251,2779.412861
2,2024-10-09T23:00:00Z,0.0,2956.745251,2779.412861
3,2024-10-09T23:30:00Z,0.0,2956.745251,2779.348132
4,2024-10-10T00:00:00Z,0.0,2956.745251,2779.348110


In [22]:
params =  {
        "timestamp_utc": "2021-10-10T23:00:00Z",
        "settlement_date": "2023-10-11",
        "settlement_period": 1,
        "boundary": "N",
        "publish_time_utc": "2021-10-10T00:15:00Z",
        "transmission_system_demand": 22683,
        "national_demand": 21370
    }



url = f"https://api.rebase.energy/challenges/data/day_ahead_demand"
resp = rebase_api_client.session.get(url, params = params)
print(resp)
resp.json()

<Response [200]>


[{'timestamp_utc': '2024-10-19T11:00:00Z',
  'settlement_date': '2024-10-19',
  'settlement_period': 25,
  'boundary': 'N',
  'publish_time_utc': '2024-10-19T10:45:00Z',
  'transmission_system_demand': 26580.0,
  'national_demand': 25100.0},
 {'timestamp_utc': '2024-10-19T11:30:00Z',
  'settlement_date': '2024-10-19',
  'settlement_period': 26,
  'boundary': 'N',
  'publish_time_utc': '2024-10-19T10:45:00Z',
  'transmission_system_demand': 26221.0,
  'national_demand': 24741.0},
 {'timestamp_utc': '2024-10-19T12:00:00Z',
  'settlement_date': '2024-10-19',
  'settlement_period': 27,
  'boundary': 'N',
  'publish_time_utc': '2024-10-19T10:45:00Z',
  'transmission_system_demand': 25840.0,
  'national_demand': 24410.0},
 {'timestamp_utc': '2024-10-19T12:30:00Z',
  'settlement_date': '2024-10-19',
  'settlement_period': 28,
  'boundary': 'N',
  'publish_time_utc': '2024-10-19T10:45:00Z',
  'transmission_system_demand': 25265.0,
  'national_demand': 23835.0},
 {'timestamp_utc': '2024-10-19T1

In [6]:
rebase_api_client

In [8]:
myDict = {'a': 1, 'b': 2}
new_data = {'c': 3, 'd': 4}

myDict.update(new_data)

print(myDict)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}


In [9]:
import comp_utils
import pandas as pd

In [10]:
rebase_api_client = comp_utils.RebaseAPI(api_key = open("DanMaLeo_key.txt").read())

In [11]:
# Nationaler Forecast für Solar- und Windproduktion
# timestamp_utc bzw. settlement_date und settlement_period (23:00 (Periode 1)- 22:30 (Periode 48)) sind hier die valid_time
solar_wind_forecast = comp_utils.RebaseAPI.get_solar_wind_forecast(rebase_api_client, "2023-10-11")
solar_wind_forecast.tail()

,timestamp_utc,settlement_date,settlement_period,solar_mw,wind_offshore_mw,wind_onshore_mw
43,2023-10-11T19:30:00Z,2023-10-11,42,0.0,5119.954667,4578.129000
44,2023-10-11T20:00:00Z,2023-10-11,43,0.0,5010.592333,4013.771000
45,2023-10-11T20:30:00Z,2023-10-11,44,0.0,5010.592333,4013.771000
46,2023-10-11T21:00:00Z,2023-10-11,45,0.0,4603.203000,3513.663667
47,2023-10-11T21:30:00Z,2023-10-11,46,0.0,4603.203000,3513.663667


In [12]:
# Forecast für day-ahead-demand
# national_demand: NDF berücksichtigt die Übertragungsverluste, aber nicht die Transformatorenlast, die Pumpspeichernachfrage oder die Interkonnektornachfrage; die Daten werden nur auf nationaler Ebene gemeldet
# transmition_system_demand: TSDF berücksichtigt die Übertragungsverluste, die Transformatorenlast, die Pumpspeichernachfrage und die Interkonnektornachfrage. Die Daten werden sowohl auf nationaler Ebene als auch auf der Ebene der Grenzgebiete (Systemzonen) gemeldet
# boundary: geografischer Bezugsbereich (https://www.arcgis.com/apps/webappviewer/index.html?id=cb3474a78df24139b1651908ff8c8975)
# keine historischen Daten verfügbar
da_demand_forecast = comp_utils.RebaseAPI.get_day_ahead_demand_forecast(rebase_api_client) # Gibt Liste zurück
da_demand_forecast = pd.DataFrame(da_demand_forecast)
da_demand_forecast

<Response [200]>


,timestamp_utc,settlement_date,settlement_period,boundary,publish_time_utc,transmission_system_demand,national_demand
0,2024-10-13T16:30:00Z,2024-10-13,36,N,2024-10-13T16:15:00Z,35246.0,34397.0
1,2024-10-13T17:00:00Z,2024-10-13,37,N,2024-10-13T16:15:00Z,35371.0,34500.0
2,2024-10-13T17:30:00Z,2024-10-13,38,N,2024-10-13T16:15:00Z,35746.0,34800.0
3,2024-10-13T18:00:00Z,2024-10-13,39,N,2024-10-13T16:15:00Z,35341.0,34396.0
4,2024-10-13T18:30:00Z,2024-10-13,40,N,2024-10-13T16:15:00Z,34054.0,33031.0
...,...,...,...,...,...,...,...
66,2024-10-15T01:30:00Z,2024-10-15,6,N,2024-10-13T16:15:00Z,22837.0,21884.0
67,2024-10-15T02:00:00Z,2024-10-15,7,N,2024-10-13T16:15:00Z,22472.0,21520.0
68,2024-10-15T02:30:00Z,2024-10-15,8,N,2024-10-13T16:15:00Z,22617.0,21666.0
69,2024-10-15T03:00:00Z,2024-10-15,9,N,2024-10-13T16:15:00Z,22272.0,21323.0


In [13]:
# Forecast für den Unterschied zwischen dem demand und der verfügbaren Generstions- Kapazität 2 bis 14 Tage im Voraus (nützlich für price forecasting)
# keine historischen Daten verfügbar
# forecast_date: valid_time
margin_forecast = comp_utils.RebaseAPI.get_margin_forecast(rebase_api_client) # Gibt Liste zurück
margin_forecast = pd.DataFrame(margin_forecast)
margin_forecast

<Response [200]>


,forecast_date,publish_time_utc,margin
0,2024-10-15,2024-10-13T16:00:00Z,17741.0
1,2024-10-16,2024-10-13T16:00:00Z,17999.0
2,2024-10-17,2024-10-13T16:00:00Z,9404.0
3,2024-10-18,2024-10-13T16:00:00Z,22108.0
4,2024-10-19,2024-10-13T16:00:00Z,22436.0
5,2024-10-20,2024-10-13T16:00:00Z,23540.0
6,2024-10-21,2024-10-13T16:00:00Z,17421.0
7,2024-10-22,2024-10-13T16:00:00Z,13842.0
8,2024-10-23,2024-10-13T16:00:00Z,11480.0
9,2024-10-24,2024-10-13T16:00:00Z,13090.0


In [14]:
# Aktuelle Wetterforecasts für Hornsea 1 von DWD
hornsea_dwd = comp_utils.RebaseAPI.get_hornsea_dwd(rebase_api_client)
hornsea_dwd.tail()

200


,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,ref_datetime,valid_datetime
88,87.259117,86.945374,86.788498,86.474747,86.317871,86.004120,87.415993,87.102249,86.945374,86.631622,...,4.098991,4.046521,3.979132,4.012907,4.047252,4.026607,4.006631,3.969180,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 00:00:00 GMT"
89,87.102249,87.259117,87.259117,87.415993,87.259117,86.945374,87.572868,87.572868,87.572868,87.572868,...,7.270855,7.270855,6.799082,6.730636,6.774969,6.774969,6.774969,6.808295,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 03:00:00 GMT"
90,86.945374,86.788498,86.160995,85.533501,85.219749,84.592255,87.572868,87.102249,86.474747,85.847244,...,8.652487,8.298055,8.579968,8.550322,8.603376,8.574615,8.518016,8.310818,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 06:00:00 GMT"
91,88.670990,88.827866,88.827866,88.984741,88.984741,88.827866,88.514114,88.514114,88.670990,88.984741,...,8.479960,8.630918,8.332643,8.390779,8.428046,8.519752,8.601488,8.740711,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 09:00:00 GMT"
92,82.396004,82.552879,82.396004,82.552879,82.866631,83.494125,83.023506,83.180382,82.552879,82.396004,...,9.610056,9.641164,10.259303,10.151072,9.970610,9.899035,9.758137,9.818524,"Sun, 13 Oct 2024 12:00:00 GMT","Fri, 18 Oct 2024 12:00:00 GMT"


In [15]:
# Aktuelle Wetterforecasts für Hornsea 1 von gfs (ncep)
hornsea_gfs = comp_utils.RebaseAPI.get_hornsea_gfs(rebase_api_client)
hornsea_gfs.tail()

200


,RelativeHumidity_latitude_53.59_longitude_1.522,RelativeHumidity_latitude_53.59_longitude_1.772,RelativeHumidity_latitude_53.59_longitude_2.022,RelativeHumidity_latitude_53.84_longitude_1.522,RelativeHumidity_latitude_53.84_longitude_1.772,RelativeHumidity_latitude_53.84_longitude_2.022,RelativeHumidity_latitude_54.09_longitude_1.522,RelativeHumidity_latitude_54.09_longitude_1.772,RelativeHumidity_latitude_54.09_longitude_2.022,Temperature_latitude_53.59_longitude_1.522,...,WindSpeed_latitude_53.59_longitude_1.772,WindSpeed_latitude_53.59_longitude_2.022,WindSpeed_latitude_53.84_longitude_1.522,WindSpeed_latitude_53.84_longitude_1.772,WindSpeed_latitude_53.84_longitude_2.022,WindSpeed_latitude_54.09_longitude_1.522,WindSpeed_latitude_54.09_longitude_1.772,WindSpeed_latitude_54.09_longitude_2.022,ref_datetime,valid_datetime
204,89.881737,89.159149,88.581085,90.026253,89.592705,89.303665,90.459808,90.170769,89.881737,12.386841,...,7.854890,7.415580,8.687059,8.259597,7.908551,8.912331,8.585663,8.333496,"Sun, 13 Oct 2024 06:00:00 GMT","Mon, 28 Oct 2024 18:00:00 GMT"
205,89.737221,87.135910,81.282959,90.026253,87.858498,82.294586,90.315292,88.003014,82.800392,12.303864,...,7.277773,7.188184,8.049738,7.799195,7.700066,8.491583,8.225869,8.134047,"Sun, 13 Oct 2024 06:00:00 GMT","Mon, 28 Oct 2024 21:00:00 GMT"
206,87.858498,84.534599,82.511360,88.436569,84.173309,82.728134,88.725601,84.968155,83.522980,12.303864,...,5.482445,5.618444,6.281653,6.092832,6.062668,6.823242,6.538765,6.335268,"Sun, 13 Oct 2024 06:00:00 GMT","Tue, 29 Oct 2024 00:00:00 GMT"
207,87.135910,85.979774,85.112671,88.581085,86.702362,85.907516,89.159149,87.280426,86.702362,12.054901,...,3.626097,4.505925,4.270422,4.289873,4.916445,4.743250,4.832332,5.187293,"Sun, 13 Oct 2024 06:00:00 GMT","Tue, 29 Oct 2024 03:00:00 GMT"
208,87.135910,86.485580,84.606857,88.147530,87.280426,85.184929,88.870117,88.147530,86.485580,12.303864,...,4.198579,4.703119,3.689659,4.689089,4.929153,4.349159,4.998676,5.136521,"Sun, 13 Oct 2024 06:00:00 GMT","Tue, 29 Oct 2024 06:00:00 GMT"


In [16]:
# Aktuelle Wetterforecasts für pes10
pes10_nwp = comp_utils.RebaseAPI.get_pes10_nwp(rebase_api_client,
                                               model="NCEP_GFS")
pes10_nwp

200


,valid_datetime,ref_datetime,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-13T06:00:00Z,2024-10-13T06:00:00Z,0.00,0.84,5.70,0,52.487256,0.401245
1,2024-10-13T07:00:00Z,2024-10-13T06:00:00Z,35.88,0.75,5.18,0,52.487256,0.401245
2,2024-10-13T08:00:00Z,2024-10-13T06:00:00Z,123.85,0.57,6.41,0,52.487256,0.401245
3,2024-10-13T09:00:00Z,2024-10-13T06:00:00Z,215.33,1.00,7.69,0,52.487256,0.401245
4,2024-10-13T10:00:00Z,2024-10-13T06:00:00Z,295.25,1.00,8.47,0,52.487256,0.401245
...,...,...,...,...,...,...,...,...
204,2024-10-28T18:00:00Z,2024-10-13T06:00:00Z,0.00,0.98,10.06,19,52.078072,-0.011349
205,2024-10-28T21:00:00Z,2024-10-13T06:00:00Z,0.00,1.00,9.01,19,52.078072,-0.011349
206,2024-10-29T00:00:00Z,2024-10-13T06:00:00Z,0.00,1.00,8.28,19,52.078072,-0.011349
207,2024-10-29T03:00:00Z,2024-10-13T06:00:00Z,0.00,1.00,8.22,19,52.078072,-0.011349


In [17]:
# Aktuelle Wetterforecasts für die Bevölkerung
demand_nwp = comp_utils.RebaseAPI.get_demand_nwp(rebase_api_client,
                                                 model="NCEP_GFS")
demand_nwp

200


,valid_datetime,ref_datetime,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-13T06:00:00Z,2024-10-13T06:00:00Z,6.04,3.40,274.55,0.00,79.60,0,51.479,-0.451
1,2024-10-13T07:00:00Z,2024-10-13T06:00:00Z,6.01,3.06,277.69,0.00,80.09,0,51.479,-0.451
2,2024-10-13T08:00:00Z,2024-10-13T06:00:00Z,7.01,2.62,303.34,0.00,75.70,0,51.479,-0.451
3,2024-10-13T09:00:00Z,2024-10-13T06:00:00Z,8.24,2.89,292.58,0.00,69.72,0,51.479,-0.451
4,2024-10-13T10:00:00Z,2024-10-13T06:00:00Z,9.23,2.85,285.99,0.00,65.02,0,51.479,-0.451
...,...,...,...,...,...,...,...,...,...,...
204,2024-10-28T18:00:00Z,2024-10-13T06:00:00Z,11.27,6.23,163.51,0.54,89.30,6,54.297,-1.533
205,2024-10-28T21:00:00Z,2024-10-13T06:00:00Z,10.81,5.87,157.80,1.31,93.64,6,54.297,-1.533
206,2024-10-29T00:00:00Z,2024-10-13T06:00:00Z,10.60,5.05,153.35,0.01,92.05,6,54.297,-1.533
207,2024-10-29T03:00:00Z,2024-10-13T06:00:00Z,9.94,4.17,143.36,0.02,94.22,6,54.297,-1.533


In [18]:
# Aktuelle Wetter Forecasts für beliebige Punkte
# forecast_date: valid_time
# keine historischen Daten verfügbar
weather_latest = comp_utils.RebaseAPI.query_weather_latest(rebase_api_client, 
                                                           model="DWD_ICON-EU",
                                                           lats='60.1, 61.2, 59.33',
                                                           lons='17.2, 13.1, 18.05',
                                                           variables='Temperature, WindSpeed, CloudCover',
                                                           query_type="GET") # query_type: GET, POST, ...
weather_latest = pd.DataFrame(margin_forecast)
weather_latest

400


,forecast_date,publish_time_utc,margin
0,2024-10-15,2024-10-13T16:00:00Z,17741.0
1,2024-10-16,2024-10-13T16:00:00Z,17999.0
2,2024-10-17,2024-10-13T16:00:00Z,9404.0
3,2024-10-18,2024-10-13T16:00:00Z,22108.0
4,2024-10-19,2024-10-13T16:00:00Z,22436.0
5,2024-10-20,2024-10-13T16:00:00Z,23540.0
6,2024-10-21,2024-10-13T16:00:00Z,17421.0
7,2024-10-22,2024-10-13T16:00:00Z,13842.0
8,2024-10-23,2024-10-13T16:00:00Z,11480.0
9,2024-10-24,2024-10-13T16:00:00Z,13090.0


In [19]:
# Funktioniert nicht
weather_latest_points = comp_utils.RebaseAPI.query_weather_latest_points(rebase_api_client,
                                                                         model="DWD_ICON-EU",
                                                                         lats='60.1, 61.2, 59.33',
                                                                         lons='17.2, 13.1, 18.05',
                                                                         variables='Temperature, WindSpeed, CloudCover')

413


KeyError: 0

In [ ]:
# Funktioniert nicht
weather_latest_points = comp_utils.RebaseAPI.query_weather_latest_grid(rebase_api_client,
                                                                       model="DWD_ICON-EU",
                                                                       lats='60.1, 61.2, 59.33',
                                                                       lons='17.2, 13.1, 18.05',
                                                                       variables='Temperature, WindSpeed, CloudCover')

In [13]:
# Konvertiert einen Dataframe zu einem Dataset
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_dwd_Hornsea1

200


<xarray.Dataset> Size: 162kB
Dimensions:            (ref_datetime: 1, valid_datetime: 93, longitude: 6,
                        latitude: 6)
Coordinates:
  * ref_datetime       (ref_datetime) object 8B 1728280800000000000
  * valid_datetime     (valid_datetime) object 744B 1728280800000000000 ... 1...
  * longitude          (longitude) object 48B '1.702' '1.767' ... '2.027'
  * latitude           (latitude) object 48B '53.77' '53.84' ... '54.03' '54.1'
Data variables:
    RelativeHumidity   (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    Temperature        (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindDirection:100  (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed          (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...
    WindSpeed:100      (ref_datetime, valid_datetime, longitude, latitude) float64 27kB ...

In [27]:
# Submit Methoden:
# day_ahead_market_times(today_date=pd.to_datetime('today'))
# comp_utils.prep_submission_in_json_format(weather_data)
# comp_utils.RebaseAPI.submit(rebase_api_client, data)

In [20]:
import comp_utils
rebase_api_client = comp_utils.RebaseAPI(api_key=open("DanMaLeo_key.txt").read())

In [21]:
import comp_utils


def create_submission_data(original_data, wind_prediction, solar_prediction, api_key_file:str = "team_key.txt"):
   
    submission_data = pd.DataFrame({"datetime":comp_utils.day_ahead_market_times()})

    prediction = dict()

    for key in wind_prediction.keys():
        prediction[key] = wind_prediction[key] + solar_prediction[key]

    # get the datetime values from the data which was used to predict the forecast
    index = original_data[-prediction["0.1"].shape[0]:].index
    # merge the predictions with the timestamps to make a prediction
    forecasts = pd.DataFrame(prediction, index = index).reset_index().merge(submission_data, left_on = "dtm", right_on = "datetime")
    forecasts.columns = ["dtm", "10", "20", "30", "40", "50", "60", "70", "80", "90", "datetime"]

    submissions = list()

    for index, row in forecasts.iterrows():
        submissions.append({"timestamp": str(pd.to_datetime(row["dtm"])).replace(" ", "T"),
                            "market_bid": row["50"],
                            "probalistic_forecast": row[["10", "20", "30", "40", "50", "60", "70", "80", "90"]].to_dict()})
        
    return {"market_day": date, "submissions": submissions}


#create_submission_data(merged_hornsea, wind_pred, solar_pred, "2024-05-17")

### AutoSubmitter Example

In [1]:
from model_utils import *
from Preprocessing import FeatureEngineerer

merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")

feature_engineerer_wind = FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe=['unavailabilityType', 'affectedUnit'])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea, deployment = False)

feature_engineerer_solar = FeatureEngineerer(label = "Solar_MWh_credit")
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

wind_model = LGBMRegressorModel(feature_engineerer_wind, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="lgbm_model_wind", load_pretrained=True)
solar_model = LGBMRegressorModel(feature_engineerer_solar, quantiles=np.round(np.arange(0.1, 1.0, 0.1), 1), model_save_dir="lgbm_model_solar", load_pretrained=True)

from submissions import AutoSubmitter

submitter = AutoSubmitter(hornsea_model = wind_model, pes_model = solar_model)
submitter.fetch_data().prepare_data().predict().prepare_submission()

Loaded pretrained Quantile Regressor model for quantile 0.1 from lgbm_model_wind\lgbm_model_quantile_0.1.pkl
Loaded pretrained Quantile Regressor model for quantile 0.2 from lgbm_model_wind\lgbm_model_quantile_0.2.pkl
Loaded pretrained Quantile Regressor model for quantile 0.3 from lgbm_model_wind\lgbm_model_quantile_0.3.pkl
Loaded pretrained Quantile Regressor model for quantile 0.4 from lgbm_model_wind\lgbm_model_quantile_0.4.pkl
Loaded pretrained Quantile Regressor model for quantile 0.5 from lgbm_model_wind\lgbm_model_quantile_0.5.pkl
Loaded pretrained Quantile Regressor model for quantile 0.6 from lgbm_model_wind\lgbm_model_quantile_0.6.pkl
Loaded pretrained Quantile Regressor model for quantile 0.7 from lgbm_model_wind\lgbm_model_quantile_0.7.pkl
Loaded pretrained Quantile Regressor model for quantile 0.8 from lgbm_model_wind\lgbm_model_quantile_0.8.pkl
Loaded pretrained Quantile Regressor model for quantile 0.9 from lgbm_model_wind\lgbm_model_quantile_0.9.pkl
Loaded pretrained Q

Fetched weather data does not appear to contain all relevant timestamps to compute 6-hour rolling interval features. Model predictions might be inaccurate.


200


Fetched weather data does not appear to contain all relevant timestamps to compute 6-hour rolling interval features. Model predictions might be inaccurate.


200


Fetched weather data does not appear to contain all relevant timestamps to compute 6-hour rolling interval features. Model predictions might be inaccurate.


200


Fetched weather data does not appear to contain all relevant timestamps to compute 6-hour rolling interval features. Model predictions might be inaccurate.


200


INFO:root:Perform data cleaning on the weather data...
INFO:root:Merge weather stations...
INFO:root:Merge with outages data (REMIT)...
c:\Users\danie\Documents\AWP2_Repo\awp2\Preprocessing.py:329: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  outages_df = pd.concat([outages_df, json_data_with_date_ranges])
c:\Users\danie\Documents\AWP2_Repo\awp2\Preprocessing.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  outages_df = pd.concat([outages_df, json_data_with_date_ranges])
INFO:root:Preprocessing done!
INFO:logger:Succes

In [5]:
merged_pes[merged_pes.isna().any(axis = 1)]

,Solar_MWh_credit,Wind_MWh_credit,capacity_mwp,cloud_cover,cloud_cover_diff,cloud_cover_max_1h,cloud_cover_max_3h,cloud_cover_max_6h,cloud_cover_mean_1h,cloud_cover_mean_3h,...,temp_mean_6h,temp_min_1h,temp_min_3h,temp_min_6h,temp_next_forecast,temp_range_3h,temp_std_1h,temp_std_3h,temp_std_6h,unused_capacity_mwp
dtm,,,,,,,,,,,,,,,,,,,,,


In [3]:
"solar_down_rad" in merged_pes.columns

True

In [4]:
merged_pes.columns

Index(['Solar_MWh_credit', 'Wind_MWh_credit', 'capacity_mwp', 'cloud_cover',
       'cloud_cover_diff', 'cloud_cover_max_1h', 'cloud_cover_max_3h',
       'cloud_cover_max_6h', 'cloud_cover_mean_1h', 'cloud_cover_mean_3h',
       'cloud_cover_mean_6h', 'cloud_cover_min_1h', 'cloud_cover_min_3h',
       'cloud_cover_min_6h', 'cloud_cover_next_forecast', 'cloud_cover_std_1h',
       'cloud_cover_std_3h', 'cloud_cover_std_6h', 'cos_day', 'cos_hour',
       'cos_month', 'installed_capacity_mwp',
       'interaction_solar_down_rad_temp', 'sin_day', 'sin_hour', 'sin_month',
       'solar_down_rad', 'solar_down_rad_diff', 'solar_down_rad_max_1h',
       'solar_down_rad_max_3h', 'solar_down_rad_max_6h',
       'solar_down_rad_mean_1h', 'solar_down_rad_mean_3h',
       'solar_down_rad_mean_6h', 'solar_down_rad_min_1h',
       'solar_down_rad_min_3h', 'solar_down_rad_min_6h',
       'solar_down_rad_next_forecast', 'solar_down_rad_range_3h',
       'solar_down_rad_std_1h', 'solar_down_rad_std_3h'

In [ ]:
submitter.predictions["final"]